In [1]:
import pandas as pd
import numpy as np
from langdetect import detect
import re

In [2]:
labeled1 = pd.read_excel('manual_labels.xlsx')
labeled2 = pd.read_excel('manual_labels2.xlsx')

In [3]:
labeled2.label.value_counts()

4    639
2    192
0     75
5     56
3     30
1      8
Name: label, dtype: int64

In [4]:
labeled2.shape

(1000, 5)

In [5]:
labeled1.label.value_counts()

4    624
2    203
5     74
0     60
3     31
1      8
Name: label, dtype: int64

In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
def clean_comment(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    res = ''
    for t in tokens:
        res += wnl.lemmatize(t)+' '
    return res
def get_tokens(text):
    wnl = WordNetLemmatizer()
    deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(deacc)
    return tokens

In [7]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
import nltk
from nltk.corpus import wordnet
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

def clean_comment(text):
    wnl = WordNetLemmatizer()
    #deacc = re.sub(r'\W',' ', text)
    tokens = word_tokenize(text)
    tags = nltk.pos_tag(tokens)
    processed = []
    for (word, tag) in tags:
        wn_tag = get_wordnet_pos(tag)
        if wn_tag!='':
            processed.append(wnl.lemmatize(word,wn_tag))
        else:
            processed.append(wnl.lemmatize(word))
    return ' '.join(processed)

In [8]:
labeled = pd.concat([labeled1, labeled2],axis = 0)

In [9]:
labeled.loc[:,'cleaned'] = labeled.Review.apply(clean_comment)

In [10]:
labeled['tokens'] = labeled.Review.apply(get_tokens)

In [11]:
labeled_long = labeled[labeled.tokens.apply(len)>6]
labeled_neg = labeled[labeled.label!=4]

In [12]:
comments = pd.read_csv('../data/comments_lang.csv')
comments_en = comments[comments.lang == 'en']
comments_en.loc[:,'cleaned'] = comments_en.Review.apply(clean_comment)

/home/lsm/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/lsm/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [45]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, LassoLarsCV
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [89]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer(ngram_range = (1,3), analyzer = 'word',
                       stop_words = 'english',
                       max_features = 500,
                       min_df = 2, max_df = 0.95).fit(comments_en.cleaned)
feats = vect.transform(labeled.cleaned).toarray()
labels = labeled.label.as_matrix()

In [90]:
def eval_model(y_train,y_test,y_train_pred,y_test_pred):
    
    class_names = ['unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    
    class_names_b = ['neg', 'pos']
    print('train scores\n')
    print(classification_report(y_train, y_train_pred, target_names = class_names))
    print('test scores\n')
    print(classification_report(y_test, y_test_pred, target_names = class_names))

In [91]:
feats.shape

(2000, 500)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(feats, labels, test_size=0.2)

In [93]:
m = OneVsRestClassifier(DummyClassifier()).fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

         unknown       0.07      0.01      0.02       107
           Crash       0.00      0.00      0.00        12
Balance problems       0.21      0.05      0.08       323
 Synchronization       0.00      0.00      0.00        49
        Positive       0.61      0.58      0.60      1001
             Bug       0.05      0.26      0.09       108

     avg / total       0.43      0.39      0.40      1600

test scores

                  precision    recall  f1-score   support

         unknown       0.17      0.07      0.10        28
           Crash       0.00      0.00      0.00         4
Balance problems       0.35      0.11      0.17        72
 Synchronization       0.00      0.00      0.00        12
        Positive       0.65      0.53      0.59       262
             Bug       0.06      0.36      0.10        22

     avg / total       0.51      0.40      0.43       400



In [94]:
m = LogisticRegression(class_weight = 'balanced').fit(X_train, y_train)
y_train_pred = m.predict(X_train)
y_test_pred = m.predict(X_test)
eval_model(y_train,y_test,y_train_pred,y_test_pred)

train scores

                  precision    recall  f1-score   support

         unknown       0.60      0.86      0.71       107
           Crash       0.86      1.00      0.92        12
Balance problems       0.92      0.82      0.87       323
 Synchronization       0.92      0.94      0.93        49
        Positive       0.95      0.93      0.94      1001
             Bug       0.82      0.88      0.85       108

     avg / total       0.91      0.90      0.90      1600

test scores

                  precision    recall  f1-score   support

         unknown       0.53      0.61      0.57        28
           Crash       0.33      0.25      0.29         4
Balance problems       0.63      0.58      0.60        72
 Synchronization       0.64      0.75      0.69        12
        Positive       0.89      0.90      0.90       262
             Bug       0.42      0.36      0.39        22

     avg / total       0.78      0.78      0.78       400



In [95]:
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
def eval_classifier(input_text,model = m):
    cleaned_text = clean_comment(input_text)
    feats = vect.transform([cleaned_text])
    class_names = [#'unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']
    prediction = model.predict(feats)
    #print(class_names[prediction[0]])
    return prediction[0]
def val_score(model):
    class_names = [#'unknown',
        'Crash',
        'Balance problems',
        'Synchronization',
        'Positive',
        'Bug']    
    val_en = pd.read_excel('validation_en.xlsx')
    y_true = []
    y_pred = []
    for i in range(1,6):
        y_true.append([i]*10)
        y_pred.append(val_en.iloc[:,i-1].apply(eval_classifier))
    y_true = np.array(y_true).flatten()
    y_pred = np.array(y_pred).flatten()
    print(classification_report(y_true, y_pred, target_names = class_names))
    return y_true,y_pred
y_true,y_pred = val_score(m)

                  precision    recall  f1-score   support

           Crash       0.00      0.00      0.00         0
Balance problems       0.60      0.30      0.40        10
 Synchronization       0.44      0.70      0.54        10
        Positive       0.86      0.60      0.71        10
             Bug       0.46      0.60      0.52        10

     avg / total       0.57      0.52      0.52        50



/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 6, does not match size of target_names, 5
  .format(len(labels), len(target_names))
/home/lsm/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [97]:
eval_classifier('this dumb game crashes every time i launch it')

1